In [107]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import datetime
import time

In [108]:
Stock_list = pd.read_csv('name.csv')['name']
date = pd.read_csv('date.csv')['date']
Stock = pd.read_csv('clear.csv')
Stock.sort_values(by=['證券代號','date'] , ascending=True, axis =0 , inplace=True)
Stock = Stock.reset_index(drop=True)


In [109]:
Stock_PRE = pd.read_csv('model.csv')
finall_date = max(Stock_PRE.date)

In [110]:
Stock_PRE

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,UD5,UD60,均線糾結1,均線糾結2,三陽開泰,前高,波段前高,突破前高,均買量,R_散戶
0,1102.0,7513759.0,4346.0,2.851104e+08,37.65,38.05,37.65,37.90,0.10,11.59,...,1,1,0,0,0,0.0,0.00,0,65602.933593,0.000000e+00
1,1103.0,341772.0,120.0,4.757645e+06,13.95,13.95,13.85,13.95,0.05,4.93,...,1,1,0,0,0,0.0,0.00,0,39646.711277,3.543026e+09
2,1104.0,129477.0,94.0,2.483480e+06,19.25,19.30,19.10,19.25,0.00,11.00,...,1,1,0,0,0,0.0,0.00,0,26419.718939,2.555703e+07
3,1108.0,119000.0,36.0,8.734100e+05,7.33,7.38,7.32,7.38,0.01,0.00,...,1,1,0,0,0,0.0,0.00,0,24260.714980,1.656625e+08
4,1109.0,242580.0,74.0,3.378340e+06,14.05,14.05,13.60,14.00,0.05,6.60,...,1,1,0,0,0,0.0,0.00,0,45652.626316,-6.861716e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438131,9944.0,180245.0,141.0,3.692991e+06,20.35,20.55,20.35,20.45,0.00,60.15,...,0,0,1,1,0,31.5,20.95,0,26191.239778,6.299618e+02
438132,9945.0,41405617.0,19367.0,2.023540e+09,50.30,50.30,47.75,48.85,3.10,8.56,...,1,1,1,0,0,55.0,50.30,0,104483.906672,-8.221837e+00
438133,9946.0,1369496.0,602.0,2.442677e+07,17.90,18.00,17.60,17.70,0.20,31.05,...,1,1,0,0,0,29.2,18.10,0,40575.964160,-1.535327e+01
438134,9955.0,171422.0,140.0,3.136504e+06,18.40,18.40,18.20,18.30,0.00,0.00,...,0,0,1,1,0,23.5,20.55,0,22403.439975,1.976164e+02


In [111]:
def 前高(df,label,test_):
    i=0
    LP = 0 
    day = 0
    while i<len(df['date']):         
        try:
            if (i < 60):
                test_.append(0)
            else:
                P = max(df['最高價'][i-60:i])
                if (P > LP) :
                    test_.append(P)
                    LP = P
                    day = i
                elif (day < (i-200)) :
                    test_.append(P)
                    LP = P
                    day = i
                else :
                    test_.append(LP)
        except Exception as e:
            print(e , i)
            test_.append(0)
        i+=1

test_ = []
for i in Stock_list :
    s = Stock[Stock['證券代號'] == i].reset_index(drop=True)
    前高(s,labels[l],test_)
Stock['前高'] = test_

def 波段前高(df,label,test_):
    i=0
    LP = 0 
    while i<len(df['date']):         
        try:
            if (i < 60):
                test_.append(0)
            else:
                P = max(df['最高價'][i-30:i+1])
                test_.append(P)
        except Exception as e:
            print(e , i)
            test_.append(0)
        i+=1

test_ = []
for i in Stock_list :
    s = Stock[Stock['證券代號'] == i].reset_index(drop=True)
    波段前高(s,labels[l],test_)
Stock['波段前高'] = test_

In [112]:
Stock

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date,前高,波段前高
0,1102.0,7513759.0,4346.0,285110415.0,37.65,38.05,37.65,37.90,0.10,11.59,...,-250000.0,0.0,250000.0,-90000.0,23000.0,113000.0,-548172.0,20190215.0,0.0,0.0
1,1102.0,7025972.0,3222.0,265972841.0,38.00,38.10,37.10,37.40,0.50,11.44,...,54000.0,58000.0,4000.0,-3000.0,15000.0,18000.0,393000.0,20190218.0,0.0,0.0
2,1102.0,6763011.0,4012.0,255739107.0,37.40,38.00,37.40,37.95,0.55,11.61,...,126000.0,162000.0,36000.0,-74000.0,12000.0,86000.0,315212.0,20190219.0,0.0,0.0
3,1102.0,12381905.0,4961.0,472115302.0,38.00,38.35,37.60,38.35,0.40,11.73,...,-251000.0,88000.0,339000.0,167000.0,195000.0,28000.0,2736508.0,20190220.0,0.0,0.0
4,1102.0,12821758.0,6876.0,496478492.0,38.05,39.20,38.05,38.85,0.50,11.88,...,-40000.0,502000.0,542000.0,176000.0,353000.0,177000.0,-671876.0,20190221.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439020,9958.0,3157431.0,2603.0,336943725.0,109.00,109.50,105.00,106.00,3.50,29.86,...,-266000.0,11000.0,277000.0,-142132.0,5000.0,147132.0,-296125.0,20210324.0,145.5,123.0
439021,9958.0,2987976.0,2390.0,314347842.0,106.50,106.50,104.00,105.00,1.00,29.58,...,-157000.0,0.0,157000.0,-13000.0,8000.0,21000.0,-333000.0,20210325.0,145.5,123.0
439022,9958.0,1381688.0,1214.0,145371367.0,106.00,106.50,104.50,105.50,0.50,24.59,...,-23000.0,2000.0,25000.0,-76000.0,2000.0,78000.0,-400000.0,20210326.0,145.5,123.0
439023,9958.0,2424768.0,2109.0,254529603.0,106.00,106.50,104.50,105.50,0.00,24.59,...,-1000.0,0.0,1000.0,-2000.0,43000.0,45000.0,-19000.0,20210329.0,145.5,123.0


In [113]:
start_date = date[date[date.values == finall_date ].index + 65].squeeze()
Stock = Stock[Stock.date > start_date].reset_index(drop=True)
Stock

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date,前高,波段前高
0,1102.0,4765448.0,2692.0,207416734.0,43.10,43.90,43.00,43.85,0.90,10.27,...,-221000.0,98000.0,319000.0,-13030.0,43000.0,56030.0,1018081.0,20201216.0,44.6,44.6
1,1102.0,9280675.0,5269.0,397878063.0,43.45,43.55,42.60,42.80,1.05,10.02,...,-171265.0,277000.0,448265.0,-99011.0,8000.0,107011.0,-3353509.0,20201217.0,44.6,44.6
2,1102.0,8469245.0,2826.0,361520316.0,42.65,43.10,42.50,42.50,0.30,9.95,...,238000.0,256000.0,18000.0,-28000.0,0.0,28000.0,-2346321.0,20201218.0,44.6,44.6
3,1102.0,8378558.0,4205.0,359281272.0,42.60,43.25,42.60,43.05,0.55,10.08,...,57104.0,82104.0,25000.0,-54572.0,428.0,55000.0,-788688.0,20201221.0,44.6,44.6
4,1102.0,5455719.0,3165.0,233452324.0,42.70,43.05,42.65,42.70,0.35,10.00,...,342895.0,378000.0,35105.0,182.0,15000.0,14818.0,-2106525.0,20201222.0,44.6,44.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59704,9958.0,3157431.0,2603.0,336943725.0,109.00,109.50,105.00,106.00,3.50,29.86,...,-266000.0,11000.0,277000.0,-142132.0,5000.0,147132.0,-296125.0,20210324.0,145.5,123.0
59705,9958.0,2987976.0,2390.0,314347842.0,106.50,106.50,104.00,105.00,1.00,29.58,...,-157000.0,0.0,157000.0,-13000.0,8000.0,21000.0,-333000.0,20210325.0,145.5,123.0
59706,9958.0,1381688.0,1214.0,145371367.0,106.00,106.50,104.50,105.50,0.50,24.59,...,-23000.0,2000.0,25000.0,-76000.0,2000.0,78000.0,-400000.0,20210326.0,145.5,123.0
59707,9958.0,2424768.0,2109.0,254529603.0,106.00,106.50,104.50,105.50,0.00,24.59,...,-1000.0,0.0,1000.0,-2000.0,43000.0,45000.0,-19000.0,20210329.0,145.5,123.0


In [114]:
def 增加技術指標MA(df,day,label,test_):
    i=0    
    while i<len(df['date']):
        try:
            test_.append(sum(df[label][i-day:i])/day)
        except:
            if i > day :
                print("HI", i)
            test_.append(0)
        i+=1

In [115]:
features =['MA5', 'MA10', 'MA20','MA60', 'buy5', 'buy10','外5', '外10', '投5','投10', '自5', '三大5', '三大10' ]
labels = ['收盤價', '收盤價', '收盤價','收盤價', '成交股數', '成交股數','外資進出','外資進出', '投信進出', '投信進出', '自營進出', '三大', '三大'] 
for l in range(len(labels)):
    test_ = []
    day = int(''.join([x for x in features[l] if x.isdigit()]))
    for i in Stock_list :
        s = Stock[Stock['證券代號'] == i].reset_index(drop=True)
        增加技術指標MA(s,day,labels[l],test_)
        
    Stock[features[l]] = test_

In [116]:
start_date = date[date[date.values == finall_date ].index + 10].squeeze()
Stock = Stock[Stock.date > start_date].reset_index(drop=True)
Stock

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,MA60,buy5,buy10,外5,外10,投5,投10,自5,三大5,三大10
0,1102.0,4113800.0,2262.0,181012384.0,43.75,44.2,43.60,44.05,0.30,10.32,...,0.000000,5673254.6,6230945.9,-9771.4,-25876.3,14200.0,-84100.0,-111430.8,-237402.2,-230731.2
1,1102.0,6084343.0,3387.0,268048623.0,44.00,44.2,43.85,44.00,0.05,10.30,...,0.000000,5162682.2,5900476.6,-213424.4,571.5,-1200.0,2200.0,-73105.0,-409329.4,-147613.9
2,1102.0,6084343.0,3387.0,268048623.0,44.00,44.2,43.85,44.00,0.05,10.30,...,0.000000,5317298.0,5606156.2,-637020.8,-356018.3,-1800.0,-1800.0,-90905.0,-830447.8,-507664.7
3,1102.0,8395088.0,4100.0,374591264.0,44.20,45.0,44.15,44.35,0.35,10.39,...,0.000000,5208831.4,5542063.3,-1247273.2,-458064.4,-3000.0,-2700.0,-58547.8,-1385665.0,-631511.4
4,1102.0,11384652.0,3431.0,503541629.0,43.95,44.4,43.80,44.40,0.05,10.40,...,0.000000,5865388.2,5986712.3,-1272353.4,-495213.8,-2600.0,-9100.0,22044.8,-1296352.6,-640720.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10089,9958.0,3157431.0,2603.0,336943725.0,109.00,109.5,105.00,106.00,3.50,29.86,...,109.200000,1654202.4,1447990.3,-66800.0,-80460.0,-144000.0,-104600.0,-32000.0,-283322.8,-212093.2
10090,9958.0,2987976.0,2390.0,314347842.0,106.50,106.5,104.00,105.00,1.00,29.58,...,109.100000,2010665.8,1634804.3,-120598.6,-54259.3,-144400.0,-111800.0,-87000.0,-430624.8,-238833.8
10091,9958.0,1381688.0,1214.0,145371367.0,106.00,106.5,104.50,105.50,0.50,24.59,...,108.875000,2235955.0,1723358.8,-130998.6,-73599.3,-75000.0,-71800.0,-119400.0,-406024.8,-239673.9
10092,9958.0,2424768.0,2109.0,254529603.0,106.00,106.5,104.50,105.50,0.00,24.59,...,108.741667,2147662.8,1767833.2,-50998.6,-70899.3,-79000.0,-91800.0,-117200.0,-322824.8,-269273.9


In [117]:
def MA交叉型態2(df):
    i=0
    list_=[]   
    while i <len(df['date']):
        try:
            if(df['MA5'][i]>df['MA10'][i]):
                if(df['MA5'][i-1]<df['MA10'][i-1]):
                    list_.append(1)
                    list_.append(1)
                    i += 1
                else:
                    list_.append(0)
            else:
                if(df['MA5'][i-1]>df['MA10'][i-1]):
                    list_.append(-1)
                    list_.append(-1)
                    i += 1
                else:
                    list_.append(0)
        except Exception as e:
            print(e)
            list_.append(0)
        i+=1
    df['MA交叉型態2'] = list_[0:len(df['date'])]
     

In [118]:
def KD(df):
    i=0
    list_K = []
    list_D = []
    while i < len(df['date']):
        try:
            RSV=(float(df['收盤價'][i])-min(df['最低價'][i-9:i]))/((max(df['最高價'][i-9:i])-min(df['最低價'][i-9:i]))+0.0001)*100
            K = 2/3*list_K[i-1] + 1/3*RSV
            
            if K > 99 :
                K = 99
            elif K < 1 :
                K = 1

            list_K.append(K)
            list_D.append(2/3*list_D[i-1]+1/3*list_K[i-1])
        except Exception as e :
            list_K.append(50)
            list_D.append(50)
            if i > 9 :
                print(e)
                print (i,list_K[i-1])            
        i+=1
    df['K'] = list_K
    df['D'] = list_D

In [119]:
MA交叉型態2(Stock)

-1


In [120]:
KD(Stock)

In [121]:
#黃金交叉=1 死亡交叉=2 沒有交叉=3
def KD交叉型態(df):
    i=0
    list_ = []   
    while i<len(df['date']):
        try:
            if(df['K'][i]>df['D'][i]):
                if(df['K'][i-1]<df['D'][i-1]):
                    list_.append(1)
                else:
                    list_.append(0)
            else:
                if(df['K'][i-1]>df['D'][i-1]):
                    list_.append(-1)
                else:
                    list_.append(0)
        except Exception as e:
            list_.append(0)
            print(e)
        i+=1
    df['KD交叉型態'] = list_

In [122]:
KD交叉型態(Stock)

-1


In [123]:
#黃金交叉=1 死亡交叉=2 沒有交叉=3
def KD交叉型態2(df):
    i=0
    list_ = []   
    while i<len(df['date']):
        try:
            if(df['K'][i]>df['D'][i]):
                if(df['K'][i-1]<df['D'][i-1]):
                    list_.append(1)
                    list_.append(1)
                    list_.append(1)
                    i+=2
                else:
                    list_.append(0)
            else:
                if(df['K'][i-1]>df['D'][i-1]):
                    list_.append(-1)
                    list_.append(-1)
                    list_.append(-1)
                    i+=2
                else:
                    list_.append(0)
        except Exception as e:
            list_.append(0)
            print(e)
        i+=1
    df['KD交叉型態2'] = list_[0:len(df['date'])]

In [124]:
KD交叉型態2(Stock)

-1


In [125]:
def 漲跌(df):
    i=0
    list_ = []
    while i < len(df['date']):
        try:
            list_.append(round((float(Stock['收盤價'][i])-float(Stock['收盤價'][i-1]))/(float(Stock['收盤價'][i-1])+0.00001)*100,2))
        except Exception as e:
            list_.append(0)
            print(e)
        i+=1
    df['漲跌'] = list_

In [126]:
漲跌(Stock)

-1


In [127]:
#均線值是昨日的均線值
def 增加技術指標RSI(df):
    i=0
    list_5=[]
    while i<len(df['date']):        
        try:
            temp5=df["漲跌"][i-5:i]
            list_5.append(len(temp5[temp5>0])/(len(temp5[temp5<0])+len(temp5[temp5>0]))*100)            
        except Exception as e:
            print(e ," 5 zero", i)
            list_5.append(50)
        i+=1
    df['RSI5']=list_5

In [128]:
增加技術指標RSI(Stock)

division by zero  5 zero 0
division by zero  5 zero 1
division by zero  5 zero 2
division by zero  5 zero 3
division by zero  5 zero 4
division by zero  5 zero 7262


def RSI交叉型態(df):
    i=0
    list_ = []   
    while i<len(df['date']):
        try:
            if(df['RSI5'][i]>df['RSI10'][i]):
                if(df['RSI5'][i-1]<df['RSI10'][i-1]):
                    list_.append(1)
                else:
                    list_.append(0)
            else:
                if(df['RSI5'][i-1]>df['RSI10'][i-1]):
                    list_.append(-1)
                else:
                    list_.append(0)
        except Exception as e:
            list_.append(0)
            print(e)
        i+=1
    df['RSI交叉型態'] = list_

RSI交叉型態(Stock)

In [129]:
def Slop_2(df , feature , label):
    list_ = []
    i = 0
    while i<len(df['date']):
      
        try:
            if Stock[label][i-1] == 0 :
                list_.append(0)
                i += 1
                continue
                
            K = (Stock[label][i]-Stock[label][i-1])/(Stock[label][i-1])*10
            if K < 0 :
                list_.append(-round(K**2 , 4))
            else : 
                list_.append( round(K**2 , 4))
        except Exception as e:
            print(e)
            list_.append(0)
        i+=1
    df[feature] = list_

In [130]:
feature = ['MA5 slope','MA10 slope','MA60 slope','ratio slope']
label = ['MA5','MA10','MA60','ratio' ]
for i in range(len(label)) :
    Slop_2(Stock , feature[i] , label[i])

-1
-1
-1
-1


In [131]:
def Slope(df , feature , label):
    list_ = []
    i = 0
    while i<len(df['date']):
        try:
            K = (df[label][i]-df[label][i-1])/(df[label][i]+0.001)*100
            list_.append(round(K , 4))
        except Exception as e:
            print(e)
            list_.append(0)
        i+=1
    df[feature] = list_

In [132]:
feature = ['margin slope','short slope','三大 slope','外資5 slope','投信5 slope','自營 slope','buy5 slope','buy10 slope','金額 slope']
label = ['融資餘','融卷餘','三大5','外5','投5', '自5', 'buy5','buy10','金額5']
for i in range(len(label)) :
    Slope(Stock , feature[i] , label[i])

-1
-1
-1
-1
-1
-1
-1
-1
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5

'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5

'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5

'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5

'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5'
'金額5

In [133]:
def RSI鈍化(df):
    i = 0 
    list_=[]
    while i<len(df['date']):
        try:
            if(df['RSI5'][i]>75):
                if df['收盤價'][i] > df['MA5'][i] :
                    list_.append(1)
                else:
                    list_.append(0)
            elif(df['RSI5'][i]<20):
                if df['收盤價'][i] < df['MA5'][i] :
                    list_.append(-1)
                else:
                    list_.append(0) 
            else:
                list_.append(0)

        except Exception as e:
            print(e)
            list_.append (0)
        i+=1
    df['RSI5鈍化'] = list_

In [134]:
RSI鈍化(Stock)

In [135]:
def intercept(df , feature , label1 , label2):
    list_ = []
    i = 0
    while i<len(df['date']):
      
        try:
            K = Stock[label1][i]*Stock[label2][i]
            list_.append(K)
        except Exception as e:
            print(e)
            list_.append(0)
            
        i+=1
    df[feature] = list_

In [136]:
feature = ['外5金','投5金','自5金','融資金','融卷金']
label1 = ['收盤價','收盤價','收盤價','收盤價','收盤價']
label2 = ['外5','投5','自5','融資買','融卷賣']
for i in range(len(feature)):
    intercept(Stock , feature[i] , label1[i] ,label2[i])

feature = ['I_mb','I_外投','I_外','I_投','I_嘎卷']
label1 = ['MA5 slope','外資5 slope','外資5 slope','外資5 slope','三大 slope']
label2 = ['buy5 slope','投信5 slope','外資10 slope','外資10 slope','short slope']
for i in range(len(feature)):
    intercept(Stock , feature[i] , label1[i] ,label2[i])

In [137]:
def 紅棒(df , feature , feature2):
    list_longbar = []
    list_antenna = []
    i = 0
    while i<len(df['date']):
        try:
            if (df['漲跌'][i] > 2):
                K  = (df['最高價'][i]-df['開盤價'][i]) / df['開盤價'][i] * 100
                if K > (df['漲跌'][i]*2) :
                    list_antenna.append(1)
                    list_longbar.append(0)
                else :
                    list_antenna.append(0)
                    list_longbar.append(1)
            else :
                list_antenna.append(0)
                list_longbar.append(0)
        except Exception as e:
            print(e)
            list_antenna.append(0)
            list_longbar.append(0)
        i+=1
    df[feature] = list_longbar
    df[feature2] = list_antenna

In [138]:
紅棒(Stock , "實紅棒","紅棒天線")

In [139]:
def 黑棒(df , feature , feature2):
    list_longbar = []
    list_antenna = []
    i = 0
    while i<len(df['date']):
        try:
            if (df['漲跌'][i] < -2):
                K  = (df['最低價'][i]-df['開盤價'][i]) / df['開盤價'][i] * 100
                if K < (df['漲跌'][i]*2) :
                    list_antenna.append(1)
                    list_longbar.append(0)
                else :
                    list_antenna.append(0)
                    list_longbar.append(1)
            else :
                list_antenna.append(0)
                list_longbar.append(0)
        except Exception as e:
            print(e)
            list_antenna.append(0)
            list_longbar.append(0)
        i+=1
    df[feature] = list_longbar
    df[feature2] = list_antenna

In [140]:
黑棒(Stock , "實黑棒","黑棒天線")

In [141]:
def 避雷針(df , feature , feature2):
    list_U = []
    list_D = []
    i = 0
    while i<len(df['date']):
        try:
            K  = (df['最高價'][i]-df['開盤價'][i]) / df['開盤價'][i] * 100
            D  = (df['最低價'][i]-df['開盤價'][i]) / df['開盤價'][i] * 100
            if (K > df['漲跌'][i]*3) and (K > 1.5):
                list_U.append(1)
                list_D.append(0)
            elif ( D < df['漲跌'][i]*3) and (D < -1.5) : 
                list_U.append(0)
                list_D.append(1)
            else :
                list_U.append(0)
                list_D.append(0)
        except Exception as e:
            print(e)
            list_U.append(0)
            list_D.append(0)
        i+=1
    df[feature] = list_U
    df[feature2] = list_D

In [142]:
避雷針(Stock , "上避雷針","下避雷針")

In [143]:
def 連續(df , feature ):
    list_U = []
    i = 0
    K = 0
    while i<len(df['date']):
        try:
            if ((df['實紅棒'][i] ==1) or (df['紅棒天線'][i] ==1) ):
                if K < 0 :
                    K = 0
                K += 1
                list_U.append(K)
                
            elif ((df['實黑棒'][i] ==1) or (df['黑棒天線'][i] ==1) ):
                if K > 0 :
                    K = 0
                K -= 1
                list_U.append(K)
            else :
                if list_U[-1] == K:
                    list_U.append(K)
                    K = 0
                else:
                    K = 0
                    list_U.append(K)
                
        except Exception as e:
            print(e)
            list_U.append(0)
        i+=1
    df[feature] = list_U

In [144]:
連續(Stock , "連漲跌")

list index out of range


In [145]:
Stock[-40:]

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,自5金,融資金,融卷金,實紅棒,紅棒天線,實黑棒,黑棒天線,上避雷針,下避雷針,連漲跌
10054,9945.0,5743364.0,2889.0,2.557477e+08,44.60,44.90,44.15,44.85,0.30,9.71,...,-1012192.74,17536.35,44.85,0,0,0,0,0,0,0
10055,9945.0,5638455.0,2909.0,2.534088e+08,44.95,45.20,44.60,45.20,0.35,9.78,...,-803131.68,18984.00,0.00,0,0,0,0,0,0,0
10056,9945.0,7946100.0,3849.0,3.550771e+08,45.20,45.35,44.30,44.40,0.80,9.61,...,-1605876.96,30058.80,399.60,0,0,0,0,0,0,0
10057,9945.0,8518411.0,4044.0,3.833000e+08,44.75,45.35,44.40,45.20,0.80,9.78,...,-557523.92,78874.00,406.80,0,0,0,0,0,0,0
10058,9945.0,6699268.0,3224.0,3.050817e+08,45.25,45.80,45.25,45.75,0.55,9.90,...,-898090.80,37103.25,228.75,0,0,0,0,0,0,0
10059,9945.0,41405617.0,19367.0,2.023540e+09,50.30,50.30,47.75,48.85,3.10,8.56,...,-309836.01,112159.60,1367.80,1,0,0,0,0,1,1
10060,9945.0,11268114.0,6173.0,5.484687e+08,48.60,49.25,48.10,48.90,0.05,8.56,...,-1863843.06,28704.30,440.10,0,0,0,0,0,0,1
10061,9946.0,1998737.0,687.0,3.295156e+07,16.00,16.90,15.90,16.40,0.40,9.37,...,-3280.00,918.40,0.00,0,0,1,0,1,0,-1
10062,9946.0,587708.0,319.0,9.608119e+06,16.40,16.45,16.20,16.45,0.05,28.86,...,-13160.00,197.40,0.00,0,0,0,0,0,0,-1
10063,9946.0,641513.0,355.0,1.056130e+07,16.45,16.60,16.35,16.50,0.05,28.95,...,0.00,247.50,0.00,0,0,0,0,0,0,0


In [146]:
def distance(df , feature ,label1 , label2):
    list_ = []
    i = 0
    while i<len(df['date']):
        
        try:
            K = (df[label1][i] - df[label2][i])/(df[label1][i]+0.001)
            list_.append(K)
        except Exception as e:
            print(e)
            list_.append(0)
        i+=1
    df[feature] = list_

In [147]:
feature = ['D5','D60','D三大','D_upper_day']
label1 = ['收盤價','收盤價','三大5','收盤價']
label2 = ['MA5','MA60','三大10','最高價']
for i in range(len(feature)):
    distance(Stock , feature[i] , label1[i] ,label2[i])

In [148]:
def Updown_state(df , feature ,label1 , label2):
    list_ = []
    i = 0
    while i<len(df['date']):
        
        try:
            if Stock[label1][i] > (Stock[label2][i] * 1.01):
                list_.append(1)
            elif Stock[label1][i] < (Stock[label2][i] * 0.95):
                list_.append(-1)
            else:
                list_.append(0)
        except Exception as e:
            print(e)
            list_.append(0)
        i+=1
    df[feature] = list_

feature = ['UD5','UD60']
label1 = ['收盤價','收盤價']
label2 = ['MA5','MA60']
for i in range(len(feature)):
    Updown_state(Stock , feature[i] , label1[i] ,label2[i])

In [149]:
def 均線糾結(df):
    i = 0 
    list_=[]
    list_2 = [] 
    while i<len(df['date']):
        try:
            MA5 = df['MA5'][i]
            MA10 = df['MA10'][i]
            MA20 = df['MA20'][i]
            MA60 = df['MA60'][i]
            P = df['收盤價'][i]
            d = min(MA5,MA10,MA20) / max(MA5,MA10,MA20) 
            d2 = min(MA5,MA10,MA20,MA60) / max(MA5,MA10,MA20,MA60) 
            if(d >0.96) :
                list_.append(1)
            else:
                list_.append(0)
            if(d2 >0.95) :
                list_2.append(1)
            else:
                list_2.append(0)
                
        except Exception as e:
            if (max(MA5,MA10,MA20,MA60) !=0) :
                print(e)
            list_.append (0)
        i+=1
    df['均線糾結1'] = list_
    df['均線糾結2'] = list_2

In [150]:
均線糾結(Stock)

In [151]:
Stock[-10:]

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,下避雷針,連漲跌,D5,D60,D三大,D_upper_day,UD5,UD60,均線糾結1,均線糾結2
10084,9958.0,1375114.0,994.0,155739538.0,113.0,114.0,112.5,114.0,2.0,32.11,...,0,1,0.013158,0.999991,-0.046245,0.000000,1,1,1,0
10085,9958.0,1861530.0,1466.0,212116370.0,114.5,115.5,113.0,113.5,0.5,31.97,...,0,0,0.007929,0.999991,0.727070,-0.017621,0,1,1,0
10086,9958.0,1823149.0,1435.0,203911692.0,113.5,113.5,111.0,111.5,2.0,31.41,...,0,0,-0.012556,0.999991,0.095873,-0.017937,0,1,1,0
10087,9958.0,1315831.0,1056.0,145946246.0,112.0,112.0,110.0,111.0,0.5,31.27,...,1,0,-0.015315,0.015615,0.396016,-0.009009,0,1,1,1
10088,9958.0,1895388.0,1481.0,207981208.0,112.5,112.5,109.0,109.5,1.5,30.85,...,0,0,-0.026484,0.002587,0.252825,-0.027397,0,0,1,1
10089,9958.0,3157431.0,2603.0,336943725.0,109.0,109.5,105.0,106.0,3.5,29.86,...,0,-1,-0.055660,-0.030188,0.251408,-0.033019,-1,0,1,1
10090,9958.0,2987976.0,2390.0,314347842.0,106.5,106.5,104.0,105.0,1.0,29.58,...,0,-1,-0.050476,-0.039047,0.445378,-0.014286,0,0,1,1
10091,9958.0,1381688.0,1214.0,145371367.0,106.0,106.5,104.5,105.5,0.5,24.59,...,0,0,-0.029384,-0.031990,0.409706,-0.009479,0,0,1,1
10092,9958.0,2424768.0,2109.0,254529603.0,106.0,106.5,104.5,105.5,0.0,24.59,...,0,0,-0.018009,-0.030726,0.165882,-0.009479,0,0,1,1
10093,9958.0,2657057.0,1946.0,282041423.0,105.5,108.5,104.0,108.5,3.0,25.29,...,0,1,0.020276,-0.000998,0.203074,0.000000,1,0,0,1


In [152]:
def 三陽開泰 (df):
    i = 0
    list_ = []
    while i<len(df['date']):
        try:
            if(df['均線糾結1'][i] == 1) :
                P = max(df.MA5[i],df.MA10[i],df.MA20[i])
                if ((df['收盤價'][i] > P) & (df['收盤價'][i] > df['開盤價'][i]) ):
                    if (max(list_[-7:-1])!= 1):
                        list_.append(1)
                    else:
                        list_.append(0)
                else :
                    list_.append(0)
            else :
                list_.append(0)
        except Exception as e:
            print(e , i)
            list_.append(0)
            
        i+=1
    df['三陽開泰'] = list_

In [153]:
三陽開泰 (Stock)

max() arg is an empty sequence 0


In [154]:
def 突破前高(df):
    i = 0
    list_ = []
    while i<len(df['date']):
        try:
            if((df['前高'][i]!=0) & (df['收盤價'][i] > df['前高'][i])):
                list_.append(1)
            else :
                list_.append(0)
        except Exception as e:
            print(e , i)
            list_.append(0)
            
        i+=1
    df['突破前高'] = list_

In [155]:
突破前高(Stock)

In [156]:
def 突破整理(df):
    i = 0
    list_ = []
    while i<len(df['date']):
        try:
            if((df['前高'][i]!=0) & (df['收盤價'][i] > df['前高'][i])):
                if (max(list_[-10:]) != 1):
                    list_.append(1)
                else :
                    list_.append(0)
            else :
                list_.append(0)
        except Exception as e:
            print(e , i)
            list_.append(0)
            
        i+=1
    df['突破前高'] = list_

In [157]:
def Ratio(df , feature , label1 , label2):
    list_ = []
    i = 0
    while i<len(df['date']):
      
        try:
            K = Stock[label1][i] / (Stock[label2][i]+0.001)
            list_.append(K)
        except Exception as e:
            print(e)
            list_.append(0)
            
        i+=1
    df[feature] = list_

In [158]:
feature = ['均買量','R_散戶']
label1 = ['成交金額','三大 slope']
label2 = ['成交筆數','margin slope']
for i in range(len(feature)):
    Ratio(Stock , feature[i] , label1[i] ,label2[i])

In [159]:
Stock_new = Stock[Stock.date > finall_date].reset_index(drop = True)
Stock_new

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,D三大,D_upper_day,UD5,UD60,均線糾結1,均線糾結2,三陽開泰,突破前高,均買量,R_散戶
0,1102.0,12438302.0,3427.0,581843757.0,46.85,47.00,46.60,46.80,0.00,10.96,...,-1.333333,-0.004273,1,1,1,0,0,0,169782.196445,339.144957
1,1103.0,11381920.0,3755.0,230867872.0,19.90,20.60,19.85,20.50,1.25,7.48,...,1.141777,-0.004878,1,1,1,0,0,1,61482.772441,-4.997672
2,1104.0,1615174.0,658.0,36428096.0,22.35,22.65,22.35,22.60,0.30,11.83,...,0.779314,-0.002212,1,1,1,1,1,0,55361.763888,-2.603912
3,1108.0,1081220.0,408.0,13642613.0,12.70,12.70,12.55,12.55,0.05,12.94,...,1.911704,-0.011951,0,1,1,1,0,0,33437.695006,468.095662
4,1109.0,1055722.0,410.0,23635766.0,22.35,22.70,22.05,22.40,0.40,7.62,...,0.496241,-0.013392,1,1,0,0,0,1,57648.069151,-2.293433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,9944.0,128366.0,130.0,2624380.0,20.35,20.55,20.35,20.55,0.10,29.78,...,-1.963285,0.000000,1,0,1,1,0,0,20187.383174,56.401931
885,9945.0,11268114.0,6173.0,548468677.0,48.60,49.25,48.10,48.90,0.05,8.56,...,-0.958061,-0.007157,1,1,1,0,1,0,88849.601191,-938.623150
886,9946.0,2069923.0,1113.0,34942249.0,17.50,17.70,16.50,16.60,1.10,29.12,...,0.077085,-0.066261,-1,1,0,0,0,0,31394.624982,-0.567789
887,9955.0,95448.0,107.0,1749797.0,18.60,18.60,18.25,18.30,0.00,0.00,...,3.513048,-0.016393,0,0,1,1,0,0,16353.090158,-7.918892


In [160]:
Stock = Stock_PRE.append(Stock_new)
Stock.sort_values(by=['證券代號','date'] , ascending=True, axis =0 , inplace=True)
Stock = Stock.reset_index(drop=True)

In [161]:
Stock_index = pd.read_csv('Market_index.csv')
Stock_index.sort_values(by=['Index','date'] , ascending=True, axis =0 , inplace=True)
index_set = ['發行量加權股價指數' , '未含電子指數' , '電子類兩倍槓桿指數' ]
Stock_index = Stock_index.reset_index(drop=True)


features =['MA5', 'MA20' ]
labels = ['收盤', '收盤'] 
for l in range(len(labels)):
    test_ = []
    day = int(''.join([x for x in features[l] if x.isdigit()]))
    for i in index_set:
        s = Stock_index[Stock_index.Index == i].reset_index(drop=True)
        增加技術指標MA(s,day,labels[l],test_)
        
    Stock_index[features[l]] = test_

features =['MA5 slop', 'MA20 slop' ]
labels = ['MA5', 'MA20'] 
for l in range(len(labels)):
    Slope (Stock_index , features[l] , labels[l] )

features =['乖離5', '乖離20' ]
label1 = ['收盤','收盤']
label2 = ['MA5','MA20']
for i in range(len(labels)):
    distance (Stock_index , features[i] , label1[i] ,label2[i] )
    
Stock_index

-1
-1


,Index,收盤,漲跌,漲跌%,date,MA5,MA20,MA5 slop,MA20 slop,乖離5,乖離20
0,未含電子指數,15054.77,65.44,0.44,20190429,0.000,0.0000,0.0000,0.0000,1.000000,1.000000
1,未含電子指數,15107.62,52.85,0.35,20190430,0.000,0.0000,0.0000,0.0000,1.000000,1.000000
2,未含電子指數,15197.27,89.65,0.59,20190502,0.000,0.0000,0.0000,0.0000,1.000000,1.000000
3,未含電子指數,15194.64,2.63,-0.02,20190503,0.000,0.0000,0.0000,0.0000,1.000000,1.000000
4,未含電子指數,15004.13,190.51,-1.25,20190506,0.000,0.0000,0.0000,0.0000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
1493,電子類兩倍槓桿指數,38471.79,1203.58,-3.03,20210324,39746.888,39809.0295,-0.3640,-0.4633,-0.033144,-0.034759
1494,電子類兩倍槓桿指數,38542.68,70.89,0.18,20210325,39464.988,39588.9735,-0.7143,-0.5559,-0.023930,-0.027146
1495,電子類兩倍槓桿指數,39897.58,1354.90,3.52,20210326,39105.166,39450.5400,-0.9201,-0.3509,0.019861,0.011205
1496,電子類兩倍槓桿指數,40813.52,915.94,2.30,20210329,39247.112,39320.5765,0.3617,-0.3305,0.038380,0.036580


In [162]:
Stock.columns

Index(['證券代號', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差',
       '本益比', '融資買', '融資賣', '融資餘', '融卷買', '融卷賣', '融卷餘', 'ratio', '外資進出', '外資進',
       '外資出', '投信進出', '投信進', '投信出', '自營進出', '自營進', '自營出', '避險進出', '避險進', '避險出',
       '三大', 'date', 'MA5', 'MA10', 'MA20', 'MA60', 'buy5', 'buy10', '外5',
       '外10', '投5', '投10', '自5', '三大5', '三大10', 'MA交叉型態2', 'K', 'D', 'KD交叉型態',
       'KD交叉型態2', '漲跌', 'RSI5', 'MA5 slope', 'MA10 slope', 'MA60 slope',
       'ratio slope', 'margin slope', 'short slope', '三大 slope', '外資5 slope',
       '投信5 slope', '自營 slope', 'buy5 slope', 'buy10 slope', '金額 slope',
       'RSI5鈍化', '外5金', '投5金', '自5金', '融資金', '融卷金', '實紅棒', '紅棒天線', '實黑棒',
       '黑棒天線', '上避雷針', '下避雷針', '連漲跌', 'D5', 'D60', 'D三大', 'D_upper_day', 'UD5',
       'UD60', '均線糾結1', '均線糾結2', '三陽開泰', '前高', '波段前高', '突破前高', '均買量', 'R_散戶'],
      dtype='object')

In [165]:
Stock

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,UD5,UD60,均線糾結1,均線糾結2,三陽開泰,前高,波段前高,突破前高,均買量,R_散戶
0,1102.0,7513759.0,4346.0,285110415.0,37.65,38.05,37.65,37.90,0.10,11.59,...,1,1,0,0,0,0.0,0.0,0,65602.933593,0.000000
1,1102.0,7025972.0,3222.0,265972841.0,38.00,38.10,37.10,37.40,0.50,11.44,...,1,1,0,0,0,0.0,0.0,0,82548.962896,-0.000000
2,1102.0,6763011.0,4012.0,255739107.0,37.40,38.00,37.40,37.95,0.55,11.61,...,1,1,0,0,0,0.0,0.0,0,63743.530223,-0.000000
3,1102.0,12381905.0,4961.0,472115302.0,38.00,38.35,37.60,38.35,0.40,11.73,...,1,1,0,0,0,0.0,0.0,0,95165.330948,0.000000
4,1102.0,12821758.0,6876.0,496478492.0,38.05,39.20,38.05,38.85,0.50,11.88,...,1,1,0,0,0,0.0,0.0,0,72204.540401,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439020,9958.0,3157431.0,2603.0,336943725.0,109.00,109.50,105.00,106.00,3.50,29.86,...,-1,0,1,1,0,145.5,123.0,0,129444.331754,94.078766
439021,9958.0,2987976.0,2390.0,314347842.0,106.50,106.50,104.00,105.00,1.00,29.58,...,0,0,1,1,0,145.5,123.0,0,131526.238692,68.701747
439022,9958.0,1381688.0,1214.0,145371367.0,106.00,106.50,104.50,105.50,0.50,24.59,...,0,0,1,1,0,145.5,123.0,0,119745.673191,-33.529054
439023,9958.0,2424768.0,2109.0,254529603.0,106.00,106.50,104.50,105.50,0.00,24.59,...,0,0,1,1,0,145.5,123.0,0,120687.284169,-85.339404


list_ma5 = []
list_ma20 = []
list_market = []
list_5D = []
list_20D = []
Market = Stock_index[(Stock_index.Index =='發行量加權股價指數' )]
for i in range(len(Stock)):
    day = Stock.date[i].squeeze()
    market = Market[(Market.date == int(day))]
    list_ma5.append(market['MA5 slop'].squeeze())
    list_ma20.append(market['MA20 slop'].squeeze())
    list_market.append(market.漲跌.squeeze())
    list_5D.append(market.乖離5.squeeze())
    list_20D.append(market.乖離20.squeeze())
    

Stock['大盤漲跌'] = list_market
Stock['大盤5MA'] = list_ma5
Stock['大盤20MA'] = list_ma20
Stock['大盤5乖離'] = list_5D
Stock['大盤20乖離'] = list_20D

In [166]:
Stock.to_csv('model.csv', index=False)